## Iterators and Iterables

this code comes from following tutorial https://realpython.com/python-iterators-iterables/

In [2]:
times = 0
while times < 3:
    print('Hello')
    times += 1

Hello
Hello
Hello


In [3]:
numbers = [1,2,3,4,5]
for number in numbers:
    print(number)

1
2
3
4
5


In [4]:
class SequenceIterator:
    def __init__(self, sequence):
        self._sequence = sequence 
        self._index = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        if self._index < len(self._sequence):
            item = self._sequence[self._index]
            self._index += 1
            return item
        else:
            raise StopIteration
        

In [5]:
for item in SequenceIterator([1, 2, 3, 4]):
    print(item)

1
2
3
4


In [6]:
# How for loops work internally
sequence = SequenceIterator([1, 2, 3, 4])

iterator = sequence.__iter__()
while True:
    try:
        item = iterator.__next__()
    except StopIteration:
        break
    else:
        # The loop's code block goes here...
        print(item)

1
2
3
4


In [7]:
class SquareIterator:
    def __init__(self, sequence):
        self._sequence = sequence
        self._index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self._index < len(self._sequence):
            square = self._sequence[self._index] ** 2
            self._index += 1
            return square
        else:
            raise StopIteration

In [8]:
for square in SquareIterator([1, 2, 3, 4, 5]):
    print(square)

1
4
9
16
25


In [9]:
class FibonacciIterator:
    def __init__(self, stop=10):
        self._stop = stop
        self._index = 0
        self._current = 0
        self._next = 1

    def __iter__(self):
        return self
    
    def __next__(self):
        if self._index < self._stop:
            self._index += 1
            fib_number = self._current
            self._current, self._next = (self._next, self._current + self._next)
            return fib_number
        else:
            raise StopIteration

In [10]:
for fib_number in FibonacciIterator():
    print(fib_number)

0
1
1
2
3
5
8
13
21
34


In [11]:
class FibonacciInfIterator:
    def __init__(self):
        self._index = 0
        self._current = 0
        self._next = 1

    def __iter__(self):
        return self

    def __next__(self):
        self._index += 1
        self._current, self._next = (self._next, self._current + self._next)
        return self._current

In [12]:
from collections.abc import Iterator

class SequenceIterator(Iterator):
    def __init__(self, sequence):
        self._sequence = sequence
        self._index = 0

    def __next__(self):
        if self._index < len(self._sequence):
            item = self._sequence[self._index]
            self._index += 1
            return item
        else:
            raise StopIteration

In [13]:
for number in SequenceIterator([1, 2, 3, 4]):
    print(number)

1
2
3
4


#### Generators

In [14]:
def sequence_generator(sequence):
    for item in sequence:
        yield item

In [15]:
sequence_generator([1, 2, 3, 4])

<generator object sequence_generator at 0x7fa3b652d9a0>

In [16]:
for number in sequence_generator([1, 2, 3, 4]):
    print(number)

1
2
3
4


In [17]:
[item for item in [1, 2, 3, 4]]

[1, 2, 3, 4]

In [18]:
(item for item in [1, 2, 3, 4])

<generator object <genexpr> at 0x7fa3b652de00>

In [19]:
generator_expression = (item for item in [1, 2, 3, 4])

In [20]:
for item in generator_expression:
    print(item)

1
2
3
4


In [21]:
def square_generator(sequence):
    for item in sequence:
        yield item**2

In [22]:
for square in square_generator([1, 2, 3, 4, 5]):
    print(square)

1
4
9
16
25


In [23]:
def fibonacci_generator(stop=10):
    current_fib, next_fib = 0, 1
    for _ in range(0, stop):
        fib_number = current_fib
        current_fib, next_fib = (
            next_fib, current_fib + next_fib
        )
        yield fib_number

In [25]:
list(fibonacci_generator())

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

#### Data processing pipeline

In [ ]:
def to_square(numbers):
    return (number**2 for number in numbers)

def to_cube(numbers):
    return (number**3 for number in numbers)

def to_even(numbers):
    return (number for number in numbers if number % 2 == 0)

def to_odd(numbers):
    return (number for number in numbers if number % 2 != 0)

def to_string(numbers):
    return (str(number) for number in numbers)

In [27]:
list(to_string(to_square(to_even(range(20)))))

['0', '4', '16', '36', '64', '100', '144', '196', '256', '324']

In [28]:
class ReusableRange:
    def __init__(self, start=0, stop=None, step=1):
        if stop is None:
            stop, start = start, 0
        self._range = range(start, stop, step)
        self._iter = iter(self._range)

    def __iter__(self):
        return self

    def __next__(self):
        try:
            return next(self._iter)
        except StopIteration:
            self._iter = iter(self._range)
            raise

In [29]:
numbers = ReusableRange(10)

In [30]:
list(numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [31]:
list(numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [32]:
import asyncio
from random import randint

class AsyncIterable:
    def __init__(self, stop):
        self._stop = stop
        self._index = 0

    def __aiter__(self):
        return self

    async def __anext__(self):
        if self._index >= self._stop:
            raise StopAsyncIteration
        await asyncio.sleep(value := randint(1, 3))
        self._index += 1
        return value

In [33]:
async def main():
    async for number in AsyncIterable(4):
        print(number)

In [34]:
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop